ちゃんとCNNをかけるように練習したやつ

In [68]:
from sklearn.datasets import fetch_mldata
import tensorflow as tf
import numpy as np

In [69]:
mnist = fetch_mldata('MNIST original')
X = mnist['data']
y = mnist['target']
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
index = np.random.permutation(60000)
X_train, y_train = X_train[index], y_train[index]

In [196]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

def minibatch(X, y, batch_size):
    index = np.random.randint(0, len(X), batch_size)
    #print(index)
    X_batch, y_batch = X[index], y[index]
    return X_batch, y_batch

## 3 layers

In [410]:
n_inputs = 28 * 28
n_outputs = 10
height, width = 28, 28
channel = 1

n_conv1_map = 32
conv1_filter_size = 2
conv1_stride = 1

n_conv2_map = 64
conv2_filter_size = 2
conv2_stride = 1

n_conv3_map = 80
conv3_filter_size = 2
conv3_stride = 2
#learning_rate = 0.005

In [411]:
tf.reset_default_graph()

In [412]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
reshaped_X = tf.reshape(X, shape=(-1, height, width, channel))
y = tf.placeholder(tf.int32, shape=(None), name='y')

In [413]:
with tf.name_scope('CNN'):
    conv1 = tf.layers.conv2d(reshaped_X, filters=n_conv1_map, kernel_size=conv1_filter_size, 
                             strides=conv1_stride, padding='SAME', activation=tf.nn.relu, name="conv1")
    conv2 = tf.layers.conv2d(conv1, filters=n_conv2_map, kernel_size=conv2_filter_size,
                             strides=conv2_stride, padding='SAME', activation=tf.nn.relu, name='conv2')
    conv3 = tf.layers.conv2d(conv2, filters=n_conv3_map, kernel_size=conv3_filter_size, 
                             strides=conv3_stride, padding='SAME', activation=tf.nn.relu, name='conv3')
    reshape_conv3 = tf.reshape(conv3,shape=(-1, 14*14*n_conv3_map))
    fc = tf.layers.dense(reshape_conv3, n_conv3_map, name='fully_connect')
    tf.shape(fc)
    output = tf.layers.dense(fc, n_outputs, name='output')

In [414]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=y)
    loss = tf.reduce_mean(xentropy)

In [415]:
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

In [416]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(output, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [417]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [418]:
n_epochs = 15
batch_size= 100

In [419]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        X_test_batch, y_test_batch = minibatch(X_test, y_test, batch_size=1000)
        acc = accuracy.eval(feed_dict={X: X_test_batch,y: y_test_batch})
        print("accuracy: ", acc)
        

accuracy:  0.957
accuracy:  0.963
accuracy:  0.97
accuracy:  0.972
accuracy:  0.973
accuracy:  0.971
accuracy:  0.979
accuracy:  0.973
accuracy:  0.984
accuracy:  0.971
accuracy:  0.984
accuracy:  0.973
accuracy:  0.981
accuracy:  0.981
accuracy:  0.983


## 2 layers

In [420]:
n_inputs = 28 * 28
n_outputs = 10
height, width = 28, 28
channel = 1

n_conv1_map = 32
conv1_filter_size = 2
conv1_stride = 1

n_conv2_map = 64
conv2_filter_size = 2
conv2_stride = 1

learning_rate = 0.005

In [421]:
tf.reset_default_graph()

In [422]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
reshaped_X = tf.reshape(X, shape=(-1, height, width, channel))
y = tf.placeholder(tf.int32, shape=(None), name='y')

In [423]:
with tf.name_scope('CNN'):
    conv1 = tf.layers.conv2d(reshaped_X, filters=n_conv1_map, kernel_size=conv1_filter_size, 
                             strides=conv1_stride, padding='SAME', activation=tf.nn.relu, name="conv1")
    conv2 = tf.layers.conv2d(conv1, filters=n_conv2_map, kernel_size=conv2_filter_size,
                             strides=conv2_stride, padding='SAME', activation=tf.nn.relu, name='conv2')
    reshape_conv2 = tf.reshape(conv2,shape=(-1, 28*28*64))
    fc = tf.layers.dense(reshape_conv2, 64, name='fully_connect')
    tf.shape(fc)
    output = tf.layers.dense(fc, n_outputs, name='output')

In [424]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=y)
    loss = tf.reduce_mean(xentropy)

In [425]:
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

In [426]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(output, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [427]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [428]:
n_epochs = 15
batch_size= 100

In [429]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        X_test_batch, y_test_batch = minibatch(X_test, y_test, batch_size=1000)
        acc = accuracy.eval(feed_dict={X: X_test_batch,y: y_test_batch})
        print("accuracy: ", acc)
        

accuracy:  0.971
accuracy:  0.978
accuracy:  0.969
accuracy:  0.977
accuracy:  0.965
accuracy:  0.962
accuracy:  0.965
accuracy:  0.968
accuracy:  0.958
accuracy:  0.974
accuracy:  0.975
accuracy:  0.967
accuracy:  0.979
accuracy:  0.964
accuracy:  0.967


## 2 conv layers and 1 pool layer

In [430]:
n_inputs = 28 * 28
n_outputs = 10
height, width = 28, 28
channel = 1

n_conv1_map = 32
conv1_filter_size = 2
conv1_stride = 1

n_conv2_map = 64
conv2_filter_size = 2
conv2_stride = 1

learning_rate = 0.005

In [431]:
tf.reset_default_graph()

In [432]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
reshaped_X = tf.reshape(X, shape=(-1, height, width, channel))
y = tf.placeholder(tf.int32, shape=(None), name='y')

In [433]:
with tf.name_scope('CNN'):
    conv1 = tf.layers.conv2d(reshaped_X, filters=n_conv1_map, kernel_size=conv1_filter_size, 
                             strides=conv1_stride, padding='SAME', activation=tf.nn.relu, name="conv1")
    conv2 = tf.layers.conv2d(conv1, filters=n_conv2_map, kernel_size=conv2_filter_size,
                             strides=conv2_stride, padding='SAME', activation=tf.nn.relu, name='conv2')
    pool1 = tf.nn.max_pool(conv2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
    reshape_pool1 = tf.reshape(pool1,shape=(-1, 14*14*64))
    fc = tf.layers.dense(reshape_pool1, 64, name='fully_connect')
    tf.shape(fc)
    output = tf.layers.dense(fc, n_outputs, name='output')

In [434]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=y)
    loss = tf.reduce_mean(xentropy)

In [435]:
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

In [436]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(output, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [437]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [438]:
n_epochs = 15
batch_size= 100

In [439]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        X_test_batch, y_test_batch = minibatch(X_test, y_test, batch_size=1000)
        acc = accuracy.eval(feed_dict={X: X_test_batch,y: y_test_batch})
        print("accuracy: ", acc)
        

accuracy:  0.978
accuracy:  0.971
accuracy:  0.973
accuracy:  0.963
accuracy:  0.965
accuracy:  0.971
accuracy:  0.974
accuracy:  0.98
accuracy:  0.983
accuracy:  0.971
accuracy:  0.979
accuracy:  0.983
accuracy:  0.982
accuracy:  0.972
accuracy:  0.983
